In [1]:
!pip install -q huggingface_hub llama-cpp-python sentence-transformers gradio torch SpeechRecognition gTTS gradio

In [2]:
import gradio as gr
from sentence_transformers import SentenceTransformer
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import speech_recognition as sr
from gtts import gTTS
import tempfile
import os

In [3]:
model_name = "mradermacher/Raya-therapist-model-GGUF"
model_file = "Raya-therapist-model.Q4_K_M.gguf"

if not os.path.exists(model_file):
    print("Downloading model...")
    hf_hub_download(
        repo_id=model_name,
        filename=model_file,
        local_dir=".",
        resume_download=True
    )
else:
    print("Model already exists")


print("\nModel status:", "Found" if os.path.exists(model_file) else "Missing")

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Raya-therapist-model.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
try:
    
    llm = Llama(
        model_path=model_file,
        n_ctx=16384, 
        n_gpu_layers=-1,
        verbose=False
    )
    print("LLM initialized successfully!")
    
except Exception as e:
    print(f"LLM initialization failed: {str(e)}")

llama_new_context_with_model: n_ctx_per_seq (16384) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


LLM initialized successfully!


In [ ]:
# Therapist system prompt
system_prompt = (
    "You are a compassionate, empathetic, and professional therapist. "
    "Your goal is to respond supportively, clearly, and cheerfully. "
    "Do NOT include any reasoning, explanations, drafts, or thoughts. "
    "ONLY output the final response for the user, nothing else. "
    "The response should be concise, friendly, and ready to send as-is."
)


messages = [{"role": "system", "content": system_prompt}]

# Convert voice to text
def transcribe_audio(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)
        try:
            return recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            return "(Sorry, I couldn't understand that.)"
        except sr.RequestError:
            return "(Speech recognition service is unavailable.)"

# Main chatbot function
def chat_with_voice(user_text, audio_input, chat_history):
    if audio_input is not None:
        user_message = transcribe_audio(audio_input)
    else:
        user_message = user_text.strip()

    if not user_message:
        return "", None, chat_history

# Adding user message to history
    messages.append({"role": "user", "content": user_message})

# Getting AI reply
    output = llm.create_chat_completion(
        messages=messages,
        temperature=0.7,
        max_tokens=512
    )
    reply = output["choices"][0]["message"]["content"]
    messages.append({"role": "assistant", "content": reply})

# Generate audio
    import re

# Ensuring only the final sentence is kept
# Extracting only the final answer (last non-empty line)
    reply_lines = [line.strip() for line in reply.split("\n") if line.strip()]
    final_reply = reply_lines[-1] if reply_lines else "(No response generated.)"

# Generate audio
    tts =tts = gTTS(final_reply, lang="en", tld="co.in")

    temp_audio_path = tempfile.mktemp(suffix=".mp3")
    tts.save(temp_audio_path)

    chat_history.append((user_message, final_reply))
    return "", temp_audio_path, chat_history

# Building UI
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 MIND MATE")

    chatbot = gr.Chatbot(height=500)
    with gr.Row():
        text_input = gr.Textbox(label="Let's Chat")
        audio_input = gr.Audio(type="filepath", sources=["microphone"], label="🎤 Let's Talk")
    output_audio = gr.Audio(label="🔊 Therapist's Voice Reply", autoplay=True)
    with gr.Row():
        clear = gr.Button("Clear Chat")

    text_input.submit(chat_with_voice, [text_input, audio_input, chatbot], [text_input, output_audio, chatbot])
    audio_input.change(chat_with_voice, [text_input, audio_input, chatbot], [text_input, output_audio, chatbot])
    clear.click(lambda: ([], "", None), None, [chatbot, text_input, audio_input])
    
    gr.HTML("""
<p>Find Help Offline:</p>
<ul>
<li><h2><a href="https://www.amahahealth.com/" target="_blank">Amaha Health</a></h2></li>
<li><h2><a href="https://www.centerformentalhealth.in/" target="_blank">Centre for Mental Health</a></h2></li>
</ul>
""")

demo.launch()


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.
